# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770859368514                   -0.52    8.0         
  2   -2.772144343775       -2.89       -1.32    1.0    188ms
  3   -2.772170423330       -4.58       -2.50    1.0    162ms
  4   -2.772170676376       -6.60       -3.31    1.0    205ms
  5   -2.772170720791       -7.35       -3.85    2.0    178ms
  6   -2.772170722876       -8.68       -4.62    1.0    193ms
  7   -2.772170723010       -9.87       -5.15    2.0    185ms
  8   -2.772170723015      -11.29       -6.23    1.0    169ms
  9   -2.772170723015      -12.88       -6.35    2.0    198ms
 10   -2.772170723015      -13.97       -6.87    1.0    188ms
 11   -2.772170723015      -14.21       -8.24    1.0    182ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770678229175                   -0.53    9.0         
  2   -2.7

1.7735578946329391

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770832306793                   -0.52    9.0         
  2   -2.772062889170       -2.91       -1.32    1.0    153ms
  3   -2.772083105806       -4.69       -2.46    1.0    163ms
  4   -2.772083350393       -6.61       -3.17    1.0    202ms
  5   -2.772083417133       -7.18       -4.07    2.0    178ms
  6   -2.772083417656       -9.28       -4.49    1.0    184ms
  7   -2.772083417808       -9.82       -5.62    1.0    162ms
  8   -2.772083417811      -11.63       -6.14    2.0    196ms
  9   -2.772083417811      -13.26       -6.55    1.0    171ms
 10   -2.772083417811      -13.62       -8.01    1.0    210ms

Polarizability via ForwardDiff:       1.772534969665887
Polarizability via finite difference: 1.7735578946329391
